# CORA - Categorizing academic publications using getML

In this notebook, we compare getML against extant approaches in the relational learning literature on the CORA data set, which is often used for benchmarking. We demonstrate that getML outperforms the state of the art in the relational learning literature on this data set. Beyond the benchmarking aspects, this notebooks showcases getML's excellent capabilities in dealing with categorical data.

Summary:

- Prediction type: __Classification model__
- Domain: __Academia__
- Prediction target: __The category of a paper__ 
- Population size: __2708__

_Author: Dr. Patrick Urbanke_

# Background

CORA is a well-known benchmarking dataset in the academic literature on relational learning. The dataset contains 2708 scientific publications on machine learning. The papers are divided into 7 categories. The challenge is to predict the category of a paper based on the papers it cites, the papers it is cited by and keywords contained in the paper.

It has been downloaded from the [CTU Prague relational learning repository](https://relational.fit.cvut.cz/dataset/CORA) (Motl and Schulte, 2015).

### A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

### Where is this running?

Your getML live session is running inside a docker container on [mybinder.org](https://mybinder.org/), a service built by the Jupyter community and funded by Google Cloud, OVH, GESIS Notebooks and the Turing Institute. As it is a free service, this session will shut down after 10 minutes of inactivity.

# Analysis

Let's get started with the analysis and set up your session:

In [1]:
import copy
import os
from urllib import request

import numpy as np
import pandas as pd
from IPython.display import Image
import getml

getml.engine.launch()
getml.engine.set_project('cora')

getML engine is already running.

Connected to project 'cora'
http://localhost:1709/#/listprojects/cora/


## 1. Loading data

### 1.1 Download from source

We begin by downloading the data from the source file:

In [2]:
conn = getml.database.connect_mariadb(
    host="relational.fit.cvut.cz",
    dbname="CORA",
    port=3306,
    user="guest",
    password="relational"
)

conn

Connection(conn_id='default',
           dbname='CORA',
           dialect='mysql',
           host='relational.fit.cvut.cz',
           port=3306)

In [3]:
def load_if_needed(name):
    """
    Loads the data from the relational learning
    repository, if the data frame has not already
    been loaded.
    """
    if not getml.data.exists(name):
        data_frame = getml.data.DataFrame.from_db(
            name=name,
            table_name=name,
            conn=conn
        )
        data_frame.save()
    else:
        data_frame = getml.data.load_data_frame(name)
    return data_frame

In [4]:
paper = load_if_needed("paper")
cites = load_if_needed("cites")
content = load_if_needed("content")

In [5]:
paper

name,paper_id,class_label
role,join_key,categorical
0,35,Genetic_Algorithms
1,40,Genetic_Algorithms
2,114,Reinforcement_Learning
3,117,Reinforcement_Learning
4,128,Reinforcement_Learning
,...,...
2703,1154500,Case_Based
2704,1154520,Neural_Networks
2705,1154524,Rule_Learning


In [6]:
cites

name,cited_paper_id,citing_paper_id
role,join_key,join_key
0,35,887
1,35,1033
2,35,1688
3,35,1956
4,35,8865
,...,...
5424,853116,19621
5425,853116,853155
5426,853118,1140289


In [7]:
content

name,paper_id,word_cited_id
role,join_key,categorical
0,35,word100
1,35,word1152
2,35,word1175
3,35,word1228
4,35,word1248
,...,...
49211,1155073,word75
49212,1155073,word759
49213,1155073,word789


### 1.2 Prepare data for getML

getML requires that we define *roles* for each of the columns.

In [8]:
paper.set_role("paper_id", getml.data.roles.join_key)
paper.set_role("class_label", getml.data.roles.categorical)
paper

name,paper_id,class_label
role,join_key,categorical
0,35,Genetic_Algorithms
1,40,Genetic_Algorithms
2,114,Reinforcement_Learning
3,117,Reinforcement_Learning
4,128,Reinforcement_Learning
,...,...
2703,1154500,Case_Based
2704,1154520,Neural_Networks
2705,1154524,Rule_Learning


In [9]:
cites.set_role(["cited_paper_id", "citing_paper_id"], getml.data.roles.join_key)
cites

name,cited_paper_id,citing_paper_id
role,join_key,join_key
0,35,887
1,35,1033
2,35,1688
3,35,1956
4,35,8865
,...,...
5424,853116,19621
5425,853116,853155
5426,853118,1140289


We need to separate our data set into a training, testing and validation set:

In [10]:
content.set_role("paper_id", getml.data.roles.join_key)
content.set_role("word_cited_id", getml.data.roles.categorical)
content

name,paper_id,word_cited_id
role,join_key,categorical
0,35,word100
1,35,word1152
2,35,word1175
3,35,word1228
4,35,word1248
,...,...
49211,1155073,word75
49212,1155073,word759
49213,1155073,word789


The goal is to predict seven different labels. We generate a target column for each of those labels. We also have to separate the data set into a training and testing set.

In [11]:
data_full = getml.data.make_target_columns(paper, "class_label")
data_full

name,paper_id,class_label=Case_Based,class_label=Genetic_Algorithms,class_label=Neural_Networks,class_label=Probabilistic_Methods,class_label=Reinforcement_Learning,class_label=Rule_Learning,class_label=Theory
role,join_key,target,target,target,target,target,target,target
0,35,0,1,0,0,0,0,0
1,40,0,1,0,0,0,0,0
2,114,0,0,0,0,1,0,0
3,117,0,0,0,0,1,0,0
4,128,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...


In [12]:
split = getml.data.split.random(train=0.7, test=0.3, validation=0.0)
split

,
0,train
1,test
2,train
3,test
4,test
,...


In [13]:
container = getml.data.Container(population=data_full, split=split)
container.add(cites=cites, content=content, paper=paper)
container.freeze()
container

population
    subset   name    rows   type
0   test     paper    821   View
1   train    paper   1887   View

peripheral
    name       rows   type     
0   cites      5429   DataFrame
1   content   49216   DataFrame
2   paper      2708   DataFrame

## 2. Predictive modeling

We loaded the data and defined the roles and units. Next, we create a getML pipeline for relational learning.

### 2.1 Define relational model

To get started with relational learning, we need to specify the data model. Even though the data set itself is quite simple with only three tables and six columns in total, the resulting data model is actually quite complicated.

That is because the class label can be predicting using three different pieces of information:

- The keywords used by the paper
- The keywords used by papers it cites and by papers that cite the paper
- The class label of papers it cites and by papers that cite the paper

The main challenge here is that `cites` is used twice, once to connect the _cited_ papers and then to connect the _citing_ papers. To resolve this, we need two placeholders on `cites`.

In [96]:
from IPython.display import display, DisplayObject, publish_display_data

In [98]:
dh = display("aa", display_id=True)

_DataModelWidget(serialized_svg='<?xml version="1.0" standalone="no"?>\r\n<svg xmlns="http://www.w3.org/2000/s…

In [100]:
publish_display_data({"data": dm.population._widget._repr_mimebundle_(), "metadata": {}, "transient": {"display_id": dh.display_id}})

In [108]:
dm = getml.data.DataModel(paper.to_placeholder("population"))

# We need two different placeholders for cites.
dm.add(getml.data.to_placeholder(cites=[cites]*2, content=content, paper=paper))
dh = display(dm, display_id=True)

,data frames,staging table
0,population,POPULATION__STAGING_TABLE_1


In [110]:
import IPython

In [111]:
IPython.display

AttributeError: module 'IPython' has no attribute 'display_formatter'

In [81]:
DisplayObject({'application/vnd.jupyter.widget-view+json': {'version_major': 2,
  'version_minor': 0,
  'model_id': 'b7e3b125236f432e8d4b367ecb767b8a'}})

<IPython.core.display.DisplayObject object>

In [54]:
dm.population._widget._repr_mimebundle_()

{'text/plain': '_DataModelWidget(serialized_svg=\'<?xml version="1.0" standalone="no"?>\\r\\n<svg xmlns="http://www.w3.org/2000/s…',
 'application/vnd.jupyter.widget-view+json': {'version_major': 2,
  'version_minor': 0,
  'model_id': 'b7e3b125236f432e8d4b367ecb767b8a'}}

In [43]:
from IPython import get_ipython
ip = get_ipython()

In [92]:
def display_with_id(data, display_id):
    iopub = ip.kernel.iopub_socket
    session = ip.kernel.session
    # data, md = ip.display_formatter.format(obj)
    transient = {'display_id': display_id}
    content = {'data': data, 'metadata': {}, 'transient': transient}
    msg_type = 'display_data'
    session.send(iopub, msg_type, content, parent=ip.parent_header)
    
def update_with_id(data, display_id):
    iopub = ip.kernel.iopub_socket
    session = ip.kernel.session
    transient = {'display_id': display_id}
    content = {'data': data, 'metadata': {}, 'transient': transient}
    msg_type = 'update_display_data'
    session.send(iopub, msg_type, content, parent=ip.parent_header)

In [93]:
display_with_id(dm.population._widget._repr_mimebundle_(), 1)

_DataModelWidget(serialized_svg='<?xml version="1.0" standalone="no"?>\r\n<svg xmlns="http://www.w3.org/2000/s…

In [101]:
update_with_id({**dm.population._widget._repr_mimebundle_(), "text/html": "<b>aaa</b>"}, dh.display_id)

In [64]:
dh.display_id

'9c8fd43769f7092a6e5f0cc0969fef36'

In [37]:
d.update("bb")

AttributeError: 'NoneType' object has no attribute 'update'

In [73]:
dm.population.join(
    dm.cites[0],
    on=('paper_id', 'cited_paper_id')
)

ValueError: A join with the following set of parameters already exists on the placeholder 'population':

Join(right='cites',
     on=('paper_id', 'cited_paper_id'),
     time_stamps=None,
     upper_time_stamp=None,
     relationship='many-to-many',
     memory=None,
     horizon=None,
     lagged_targets=False)

Redundant joins are not allowed.

In [36]:
dm.population._to_hierarchy_tree()

{'name': 'population',
 'children': [{'name': 'cites',
   'on': 'paper_id = cited_paper_id',
   'children': [{'name': 'content', 'on': 'citing_paper_id = paper_id'},
    {'name': 'paper', 'on': 'citing_paper_id = paper_id'}]},
  {'name': 'cites',
   'on': 'paper_id = citing_paper_id',
   'children': [{'name': 'content', 'on': 'cited_paper_id = paper_id'}]}]}

In [33]:
dm.population._widget.serialized_svg

'<?xml version="1.0" standalone="no"?>\r\n<svg xmlns="http://www.w3.org/2000/svg" height="500" width="600"><defs><linearGradient id="population-gradient" x1="0" y1="0" x2="0" y2="1"><stop offset="0%" stop-color="#d54c4c" stop-opacity="1"/><stop offset="100%" stop-color="#71248e" stop-opacity="1"/></linearGradient></defs><g class="visx-group" transform="translate(300, 235)"><g class="visx-group" transform="translate(0, 0)"><path class="visx-link visx-link-radial-diagonal" d="M0,0C-18.154748419528158,55.87457033234028,55.87457033234027,18.15474841952816,111.74914066468054,36.30949683905632" stroke="#374469" stroke-width="1" fill="none"/><path class="visx-link visx-link-radial-diagonal" d="M0,0C-18.154748419528158,55.87457033234028,55.87457033234027,18.15474841952816,111.74914066468054,36.30949683905632" stroke="transparent" stroke-width="10" fill="none"/></g><g class="visx-group" transform="translate(0, 0)"><path class="visx-link visx-link-radial-diagonal" d="M0,0C-18.154748419528158,55.

In [25]:
dm.cites[0].join(
    dm.content,
    on=('citing_paper_id', 'paper_id')
)

In [26]:
dm.cites[0].join(
    dm.paper,
    on=('citing_paper_id', 'paper_id'),
    relationship=getml.data.relationship.many_to_one
)

In [27]:
dm.population.join(
    dm.cites[1],
    on=('paper_id', 'citing_paper_id')
)

In [30]:
dm.cites[1].join(
    dm.content,
    on=('cited_paper_id', 'paper_id')
)

In [21]:
dm

,data frames,staging table
0,population,POPULATION__STAGING_TABLE_1
1,"cites, paper",CITES__STAGING_TABLE_2
2,cites,CITES__STAGING_TABLE_3
3,content,CONTENT__STAGING_TABLE_4


In [69]:
from IPython.display import display_html, display, SVG

In [85]:
dm = getml.data.DataModel(paper.to_placeholder("population"))

# We need two different placeholders for cites.
dm.add(getml.data.to_placeholder(cites=[cites]*2, content=content, paper=paper))

dm.population.join(
    dm.cites[0],
    on=('paper_id', 'cited_paper_id')
)

dm.cites[0].join(
    dm.content,
    on=('citing_paper_id', 'paper_id')
)

dm.cites[0].join(
    dm.paper,
    on=('citing_paper_id', 'paper_id'),
    relationship=getml.data.relationship.many_to_one
)

dm.population.join(
    dm.cites[1],
    on=('paper_id', 'citing_paper_id')
)

dm.cites[1].join(
    dm.content,
    on=('cited_paper_id', 'paper_id')
)

dm.cites[1].join(
    dm.paper,
    on=('cited_paper_id', 'paper_id'),
    relationship=getml.data.relationship.many_to_one
)

dm.population.join(
    dm.content,
    on='paper_id'
)

In [91]:
dm

,data frames,staging table
0,population,POPULATION__STAGING_TABLE_1
1,"cites, paper",CITES__STAGING_TABLE_2
2,"cites, paper",CITES__STAGING_TABLE_3
3,content,CONTENT__STAGING_TABLE_4


In [23]:
vars(dm)

{'population': population:
   columns:
   - class_label: categorical
   - paper_id: join_key
 
   joins:
   - right: 'cites'
     on: (population.paper_id, cites.cited_paper_id)
     relationship: 'many-to-many'
     lagged_targets: False
   - right: 'cites'
     on: (population.paper_id, cites.citing_paper_id)
     relationship: 'many-to-many'
     lagged_targets: False
   - right: 'content'
     on: (population.paper_id, content.paper_id)
     relationship: 'many-to-many'
     lagged_targets: False,
 'peripheral': {'cites': [cites:
     columns:
     - cited_paper_id: join_key
     - citing_paper_id: join_key
   
     joins:
     - right: 'content'
       on: (cites.citing_paper_id, content.paper_id)
       relationship: 'many-to-many'
       lagged_targets: False
     - right: 'paper'
       on: (cites.citing_paper_id, paper.paper_id)
       relationship: 'many-to-one'
       lagged_targets: False,
   cites:
     columns:
     - cited_paper_id: join_key
     - citing_paper_id: join_

In [24]:
dm.cites

[cites:
   columns:
   - cited_paper_id: join_key
   - citing_paper_id: join_key
 
   joins:
   - right: 'content'
     on: (cites.citing_paper_id, content.paper_id)
     relationship: 'many-to-many'
     lagged_targets: False
   - right: 'paper'
     on: (cites.citing_paper_id, paper.paper_id)
     relationship: 'many-to-one'
     lagged_targets: False,
 cites:
   columns:
   - cited_paper_id: join_key
   - citing_paper_id: join_key
 
   joins:
   - right: 'content'
     on: (cites.cited_paper_id, content.paper_id)
     relationship: 'many-to-many'
     lagged_targets: False
   - right: 'paper'
     on: (cites.cited_paper_id, paper.paper_id)
     relationship: 'many-to-one'
     lagged_targets: False]

In [25]:
dm.widget.data_model

AttributeError: 'DataModel' object has no attribute 'widget'

In [ ]:
dm.to_hierarchy_tree()

### 2.2 getML pipeline

<!-- #### 2.1.1  -->
__Set-up the feature learner & predictor__

We use the relboost algorithms for this problem. Because of the large number of keywords, we regularize the model a bit by requiring a minimum support for the keywords (`min_num_samples`).

In [ ]:
mapping = getml.preprocessors.Mapping()

fast_prop = getml.feature_learning.FastProp(
    loss_function=getml.feature_learning.loss_functions.CrossEntropyLoss,
    num_threads=1
)

relboost = getml.feature_learning.Relboost(
    num_features=10,
    num_subfeatures=10,
    loss_function=getml.feature_learning.loss_functions.CrossEntropyLoss,
    seed=4367,
    num_threads=1,
    min_num_samples=30
)

predictor = getml.predictors.XGBoostClassifier()

__Build the pipeline__

In [ ]:
pipe1 = getml.pipeline.Pipeline(
    tags=['fast_prop'],
    data_model=dm,
    preprocessors=[mapping],
    feature_learners=[fast_prop],
    predictors=[predictor]
)

pipe1

In [ ]:
pipe2 = getml.pipeline.Pipeline(
    tags=['relboost'],
    data_model=dm,
    feature_learners=[relboost],
    predictors=[predictor]
)

pipe2

### 2.3 Model training

In [ ]:
pipe1.check(container.train)

In [ ]:
pipe1.fit(container.train)

In [ ]:
pipe2.check(container.train)

The training process seems a bit intimidating. That is because the relboost algorithms needs to train separate models for each class label. This is due to the nature of the generated features.

In [ ]:
pipe2.fit(container.train)

### 2.4 Model evaluation

In [ ]:
pipe1.score(container.test)

In [ ]:
pipe2.score(container.test)

To make things a bit easier, we just look at our test results.

In [ ]:
pipe1.scores.filter(lambda score: score.set_used == "test")

In [ ]:
pipe2.scores.filter(lambda score: score.set_used == "test")

We take the average of the AUC values, which is also the value that appears in the getML monitor (http://localhost:1709/#/listpipelines/cora).

In [ ]:
print(np.mean(pipe1.auc))
print(np.mean(pipe2.auc))

The accuracy for multiple targets can be calculated using one of two methods. The first method is to simply take the average of the pair-wise accuracy values, which is also the value that appears in the getML monitor (http://localhost:1709/#/listpipelines/cora).

In [ ]:
print(np.mean(pipe1.accuracy))
print(np.mean(pipe2.accuracy))

However, the benchmarking papers actually use a different approach: 

- They first generate probabilities for each of the labels:

In [ ]:
probabilities1 = pipe1.predict(container.test)
probabilities2 = pipe2.predict(container.test)

- They then find the class label with the highest probability:

In [ ]:
class_label = paper.class_label.unique()

ix_max = np.argmax(probabilities1, axis=1)
predicted_labels1 = np.asarray([class_label[ix] for ix in ix_max])

ix_max = np.argmax(probabilities2, axis=1)
predicted_labels2 = np.asarray([class_label[ix] for ix in ix_max])

- They then compare that value to the actual class label:

In [ ]:
actual_labels = paper[split == "test"].class_label.to_numpy()

print("Share of accurately predicted class labels (pipe1):")
print((actual_labels == predicted_labels1).sum() / len(actual_labels))
print()
print("Share of accurately predicted class labels (pipe2):")
print((actual_labels == predicted_labels2).sum() / len(actual_labels))
print()

Since this is the method the benchmark papers use, this is the accuracy score we will report as well.

### 2.5 Studying features

__Feature correlations__

We want to analyze how the features are correlated with the target variables.

In [ ]:
TARGET_NUM = 0

In [ ]:
names, correlations = pipe2.features.correlations(target_num=TARGET_NUM)

plt.subplots(figsize=(20, 10))

plt.bar(names, correlations, color='#6829c2')

plt.title('Feature correlations with class label ' + class_label[TARGET_NUM])
plt.xlabel('Features')
plt.ylabel('Correlations')
plt.xticks(rotation='vertical')
plt.show()

__Feature importances__
 
Feature importances are calculated by analyzing the improvement in predictive accuracy on each node of the trees in the XGBoost predictor. They are then normalized, so that all importances add up to 100%.

In [ ]:
names, importances = pipe2.features.importances()

plt.subplots(figsize=(20, 10))

plt.bar(names, importances, color='#6829c2')

plt.title('Feature importances for class label ' + class_label[TARGET_NUM])
plt.xlabel('Features')
plt.ylabel('Importances')
plt.xticks(rotation='vertical')
plt.show()

__Column importances__

Because getML uses relational learning, we can apply the principles we used to calculate the feature importances to individual columns as well.

In [ ]:
names, importances = pipe2.columns.importances(target_num=TARGET_NUM)

plt.subplots(figsize=(20, 10))

plt.bar(names, importances, color='#6829c2')

plt.title('Columns importances for class label ' + class_label[TARGET_NUM])
plt.xlabel('Columns')
plt.ylabel('Importances')
plt.xticks(rotation='vertical')
plt.show()

The most important features look as follows:

In [ ]:
pipe1.features.to_sql()[pipe1.features.sort(by="importances")[0].name]

In [ ]:
pipe2.features.to_sql()[pipe2.features.sort(by="importances")[0].name]

### 2.6 Productionization

It is possible to productionize the pipeline by transpiling the features into production-ready SQL code. Please also refer to getML's `sqlite3` and `spark` modules.

In [ ]:
# Creates a folder containing the SQL code.
pipe1.features.to_sql().save("cora_pipeline")

In [ ]:
pipe1.features.to_sql(dialect=getml.pipeline.dialect.spark_sql).save("cora_spark")

### 2.7 Benchmarks

State-of-the-art approaches on this data set perform as follows:

| Approach                      | Study                      | Accuracy     | AUC     |
| :--------------------------   | :------------------------  | -----------: | ------: |
| RelF                          | Dinh et al (2012)          | 85.7%        |     --  |
| LBP                           | Dinh et al (2012)          | 85.0%        |     --  |
| EPRN                          | Preisach and Thieme (2006) | 84.0%        |     --  |
| PRN                           | Preisach and Thieme (2006) | 81.0%        |     --  |
| ACORA                         | Perlich and Provost (2006) | --           |   97.0% |


As we can see, the performance of the relboost algorithm, as used in this notebook, compares favorably to these benchmarks.

| Approach                      | Accuracy     | AUC     |
| :---------------------------- | -----------: | ------: |
| FastProp                      | 89.9%        | 98.5%   |
| Relboost                      | 89.9%        | 98.3%   |


## 3. Conclusion

In this notebook we have demonstrated that getML outperforms state-of-the-art relational learning algorithms on the CORA dataset.

## References

Dinh, Quang-Thang, Christel Vrain, and Matthieu Exbrayat. "A Link-Based Method for Propositionalization." ILP (Late Breaking Papers). 2012.

Motl, Jan, and Oliver Schulte. "The CTU prague relational learning repository." arXiv preprint arXiv:1511.03086 (2015).

Perlich, Claudia, and Foster Provost. "Distribution-based aggregation for relational learning with identifier attributes." Machine Learning 62.1-2 (2006): 65-105.

Preisach, Christine, and Lars Schmidt-Thieme. "Relational ensemble classification." Sixth International Conference on Data Mining (ICDM'06). IEEE, 2006.

# Next Steps

This tutorial benchmarked getML against academic state-of-the-art algorithms from relational learning literature and getML's qualities with respect to categorical data.

If you are interested in further real-world applications of getML, head back to the [notebook overview](welcome.md) and choose one of the remaining examples.

Here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Feature learning with Multirel](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#multirel)
* [Feature learning with Relboost](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#relboost)

# Get in contact

If you have any question schedule a [call with Alex](https://go.getml.com/meetings/alexander-uhlig/getml-demo), the co-founder of getML, or write us an [email](team@getml.com). Prefer a private demo of getML? Just contact us to make an appointment.